### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M12D20', 'UTC 14:34')


# Settings

In [2]:
MAXTASKS = 10
CANCELTASKS = 0

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9


In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,id,output_url,progress,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,22.502217,1513777967487,reduced_global_historical_combined_long_trend_...,MJ5NK74UTX7DFAPJW6F3DPCR,NaN,0.0,19.005917,41.508133,14:11:47,1513779107842,RUNNING,EXPORT_IMAGE,1513780457975
1,18.799617,1513777963895,reduced_global_historical_combined_short_mean_...,XIW5FHZQKJBCYAVUSWC5Q5VZ,https://code.earthengine.google.com/?asset=pro...,NaN,15.433167,34.232783,14:08:09,1513778889885,COMPLETED,EXPORT_IMAGE,1513780017862
2,11.676767,1513777960731,reduced_global_historical_combined_long_mean_W...,BWX2DU4IRL5LMSUH4JPKGEKH,https://code.earthengine.google.com/?asset=pro...,NaN,13.445933,25.122700,14:06:07,1513778767487,COMPLETED,EXPORT_IMAGE,1513779468093
3,12.217383,1513767540018,reduced_global_historical_combined_short_trend...,KNCUA2JFJNEH4CEJQMPQZACD,https://code.earthengine.google.com/?asset=pro...,NaN,174.770833,186.988217,13:53:46,1513778026268,COMPLETED,EXPORT_IMAGE,1513778759311
4,19.175450,1513767535651,reduced_global_historical_combined_long_trend_...,7O45GAOBJVNAWNJNUISPYT75,https://code.earthengine.google.com/?asset=pro...,NaN,174.800617,193.976067,13:53:43,1513778023688,COMPLETED,EXPORT_IMAGE,1513779174215
5,17.818683,1513767531305,reduced_global_historical_combined_short_mean_...,FPQILRRJXN4OTOURVE5ZVENA,https://code.earthengine.google.com/?asset=pro...,NaN,174.761983,192.580667,13:53:37,1513778017024,COMPLETED,EXPORT_IMAGE,1513779086145
6,25.274467,1513767526893,reduced_global_historical_combined_long_mean_W...,OPE5RSZAYPR5I2SPSS25H6FW,https://code.earthengine.google.com/?asset=pro...,NaN,164.033350,189.307817,13:42:48,1513777368894,COMPLETED,EXPORT_IMAGE,1513778885362
7,12.071767,1513767523019,reduced_global_historical_combined_short_trend...,5Z7LVQML7FHLNOYYEHE66QLR,https://code.earthengine.google.com/?asset=pro...,NaN,157.589483,169.661250,13:36:18,1513776978388,COMPLETED,EXPORT_IMAGE,1513777702694
8,21.451517,1513767518626,reduced_global_historical_combined_short_trend...,QZJBLOV72BUYM7ZZ5SPFES6L,https://code.earthengine.google.com/?asset=pro...,NaN,153.503567,174.955083,13:32:08,1513776728840,COMPLETED,EXPORT_IMAGE,1513778015931
9,21.757000,1513767514777,reduced_global_historical_combined_short_trend...,SVOZEC5UGDD2T5LJP5FIT2RG,https://code.earthengine.google.com/?asset=pro...,NaN,151.950317,173.707317,13:30:31,1513776631796,COMPLETED,EXPORT_IMAGE,1513777937216


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)